# Basic setting

In [35]:
import pandas as pd
from glob import glob
import fasttext
from gensim.models import Word2Vec, FastText
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.metrics.pairwise import cosine_similarity

# Load data

In [2]:
dict_data = {}
for i in glob('data/*.csv'):
    try:
        dict_data[i.split('/')[1].split('.csv')[0]] = pd.read_csv(i)
    except:
        dict_data[i.split('/')[1].split('.csv')[0]] = pd.read_csv(i, encoding='cp949')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df = dict_data['suggest ver2.0 (contents added)']
df

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,RST_PROPOSER,PUBL_PROPOSER,bill_content
0,PRC_A2T0Z0E9U1I6P1G3P3X9I3B7H7D9C1,2104402,독점규제 및 공정거래에 관한 법률 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,박홍근의원등12인,http://likms.assembly.go.kr/bill/coactorListPo...,박홍근,"김원이,남인순,서영석,송옥주,용혜인,윤미향,이용빈,장경태,정태호,주철현,최종윤",「독점규제 및 공정거래에 관한 법률」(이하 “공정거래법”이라 함)은 시장경제의 기본...
1,PRC_I2M0O1G0U0P5E1U8T4B6Z0R9M8G5X0,2104401,지방교육재정교부금법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"김병기,서삼석,신정훈,오영환,윤재갑,이개호,이성만,전혜숙,최형두",제안이유 및 주요내용 현행법 제3조제2항은 지방교육재정교부금 재원을 내국세 총액에...
2,PRC_C2Q0L1M0V0B5G1O8H1N6Z5V0W5W7A7,2104400,농업기계화 촉진법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,윤재갑의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,윤재갑,"강병원,김수흥,서영교,위성곤,이개호,이재정,이정문,전용기,조경태,홍문표",제안이유 및 주요내용 현행법은 농업기계의 제조·수입업자는 다른 용도의 기계와 구분...
3,PRC_F2A0Q1V0F0S5O1N8W1P6L2Z9T9D6K3,2104399,지방교부세법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"김병기,신정훈,오영환,윤재갑,윤준병,이개호,이성만,전혜숙,최형두",제안이유 및 주요내용 현행 소방공무원의 국가직 전환 이후 소방안전교부세율은 담배에...
4,PRC_C2N0L1B0M0O5S1P8T0O6I1O8S4J8O7,2104398,농수산물의 원산지 표시에 관한 법률 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,윤재갑의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,윤재갑,"강병원,김수흥,서영교,위성곤,이개호,이재정,이정문,전용기,조경태,홍문표",제안이유 및 주요내용 현행법에 따르면 식품접객업 및 집단급식소 중 대통령령령으로 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62653,000021,10025,반민족행위특별재판부부속기관조직법안,NaN,1948-11-23,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,김상덕의원외9인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유反民族行爲特別裁判部 및 特別檢察部의 附屬機關의 組織에 관한 規定을 두어 ...
62654,000022,10024,반민족행위처벌법중개정법률안,NaN,1948-11-23,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,김상덕의원외9인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유特別調査委員의 調査業務를 圖滑히 하기 위하여 調査上 必要할 境遇 司法警察...
62655,000018,10021,농지개혁법안,NaN,1948-11-13,대안반영폐기,1,http://likms.assembly.go.kr/bill/billDetail.do...,이훈구의원외31인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유 및 주요골자資料없음.資料없음.■ 기타사항第2回國會(定期會) 産業委員會(...
62656,000014,10015,한글전용에관한법률안,NaN,1948-09-29,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,권태희의원외138인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유한 나라의 國語에는 그 나라의 國民精神이 흐르고 있고 民族正氣가 內包되어...


# Install FastText

In [5]:
!wget https://github.com/facebookresearch/fastText/archive/0.2.0.zip
!unzip 0.2.0.zip
%cd fastText-0.2.0
!make

--2020-11-28 21:39:17--  https://github.com/facebookresearch/fastText/archive/0.2.0.zip
Resolving github.com (github.com)... 52.78.231.108
Connecting to github.com (github.com)|52.78.231.108|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/0.2.0 [following]
--2020-11-28 21:39:17--  https://codeload.github.com/facebookresearch/fastText/zip/0.2.0
Resolving codeload.github.com (codeload.github.com)... 13.124.75.135
Connecting to codeload.github.com (codeload.github.com)|13.124.75.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘0.2.0.zip’

0.2.0.zip               [   <=>              ]   4.10M  7.61MB/s    in 0.5s    

2020-11-28 21:39:18 (7.61 MB/s) - ‘0.2.0.zip’ saved [4304799]

Archive:  0.2.0.zip
7842495a4d64c7a3bb4339d45d6e64321d002ed8
   creating: fastText-0.2.0/
   creating: fastText-0.2.0/.circleci/
  inflating: fastText-0.2

  inflating: fastText-0.2.0/website/static/docs/en/html/functions_p.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_q.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_r.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_s.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_t.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_u.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_v.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_vars.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_w.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/functions_z.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/globals.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/globals_defs.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/globals_func.html  
  inflating: fastText-0.

  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_1.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_1.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_2.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_2.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_3.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/defines_3.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_0.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_0.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_1.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_1.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_2.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/enums_2.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/search/e

  inflating: fastText-0.2.0/website/static/docs/en/html/search/variables_f.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/splitbar.png  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1Node-members.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1Node.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1Node.js  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1entry-members.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1entry.html  
  inflating: fastText-0.2.0/website/static/docs/en/html/structfasttext_1_1entry.js  
 extracting: fastText-0.2.0/website/static/docs/en/html/sync_off.png  
 extracting: fastText-0.2.0/website/static/docs/en/html/sync_on.png  
 extracting: fastText-0.2.0/website/static/docs/en/html/tab_a.png  
 extracting: fastText-0.2.0/website/static/docs/en/html/tab_b.png  
 extracting: fastText-0.2.0/website/stati

5 warnings generated.
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops args.o dictionary.o productquantizer.o matrix.o qmatrix.o vector.o model.o utils.o meter.o fasttext.o src/main.cc -o fasttext


# Build corpus

In [8]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [13]:
df['tokenized'] = df['bill_content'].progress_apply(tokenized_mecab)

In [14]:
tokenized = ''
for i in df['tokenized'].values:
    tokenized = tokenized + "{} \n \n".format(i)

In [16]:
f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/대학 관련/연세대학교/학회관련/YBIGTA/We are Team Analytics/2020-2/공모전/국회 입법 시각화/data/corpus ver1.0.txt", 'w')
f.write(tokenized)
f.close()

In [17]:
direc ='/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/대학 관련/연세대학교/학회관련/YBIGTA/We are Team Analytics/2020-2/공모전/국회 입법 시각화/data/corpus ver1.0.txt'

corpus = [sent.strip().split(" ") for sent in open(direc).readlines()]

In [19]:
len(corpus)

125316

# Train Fasttext (iteration 5)

In [21]:
%%time
model_gs = FastText()

model_gs.build_vocab(sentences = corpus)

CPU times: user 14.1 s, sys: 2.41 s, total: 16.5 s
Wall time: 17 s


In [22]:
%%time
model_gs.train(sentences = corpus, total_examples = len(corpus), epochs=5)

CPU times: user 3min 24s, sys: 1.19 s, total: 3min 26s
Wall time: 1min 10s


In [26]:
model_gs.wv.most_similar("장애")

[('장애등급제', 0.8282562494277954),
 ('언어기능장애', 0.8254252076148987),
 ('장애자', 0.8070912957191467),
 ('장애아', 0.7930310964584351),
 ('노숙', 0.7361632585525513),
 ('장애인', 0.6990354061126709),
 ('부랑', 0.6983562707901001),
 ('장애인자립생활지원센터', 0.6741937398910522),
 ('노숙인', 0.6652418375015259),
 ('심신장애', 0.6537489891052246)]

### Save vocab and vector as dictionary

In [33]:
word_dict = {}
for vocab in tqdm(model_gs.wv.index2word):
    word_dict[vocab] = model_gs.wv[vocab]

In [34]:
word_dict

{'하': array([-1.9734501 ,  3.248476  , -3.7067134 ,  0.0462907 , -0.74886835,
         0.17566735, -1.5460613 , -3.6435237 ,  3.2225962 ,  3.19725   ,
         0.5990901 , -1.6573529 ,  0.07585255,  6.1053305 , -3.5509346 ,
        -0.40332648, -5.6750283 , -3.0338852 ,  7.4412985 ,  0.8788209 ,
         3.6216173 ,  1.0189366 , -0.5808624 , -2.2377453 ,  2.188923  ,
        -0.70342684,  1.6657395 ,  6.581273  ,  2.5759497 ,  0.82778144,
         1.6112797 ,  2.9511547 ,  0.9851726 , -4.994364  ,  2.3455658 ,
        -5.275007  , -0.76338434,  2.0447502 ,  1.0084915 ,  1.6082643 ,
        -2.4602318 ,  7.8934984 ,  0.451854  , -3.2995849 , -0.21484604,
         0.5281024 ,  2.786726  , -0.11543354,  5.3746386 ,  1.9470181 ,
         2.7245426 ,  0.62700284, -1.4141852 ,  1.7133831 , -0.16754946,
        -3.9152076 ,  2.4902124 , -8.81557   ,  1.0703881 , -3.8199148 ,
        -5.667184  , -1.2301011 ,  2.9155245 ,  4.913348  , -2.398881  ,
        -1.8167291 ,  2.4898858 ,  2.77691   ,

### Test

In [36]:
word_dict['장애']

array([-5.1489143 ,  2.2489886 ,  0.24075663,  1.29726   , -2.9464257 ,
       -1.9434977 , -0.96047735,  2.338284  , -7.002389  ,  2.4981022 ,
       -0.6782136 , -0.40938136, -1.3682386 ,  0.7017691 , -1.2781463 ,
       -1.4564855 , -1.5965434 , -3.0764973 , -1.2408832 ,  1.5223647 ,
       -0.03220039, -5.8657765 , -1.6814507 , -2.9390373 ,  0.12125233,
       -3.8581617 , -4.0783796 , -0.09586599,  0.5538427 , -0.8705998 ,
        0.69815356, -1.076978  ,  1.8447845 ,  2.7166533 , -1.475215  ,
        0.44888368,  2.2559626 ,  1.7957875 , -0.96870947, -0.20445675,
       -4.1341667 ,  3.2865095 , -1.0324218 , -1.9327573 ,  1.9156137 ,
        2.6093094 ,  2.289479  ,  6.228277  ,  1.7570204 , -3.7234852 ,
       -0.5133893 , -3.408637  , -3.741968  ,  1.6931512 ,  1.5032847 ,
       -1.1377636 , -0.3037653 , -1.5239894 ,  0.07654929, -0.34195158,
       -0.871468  , -0.55212337, -3.1577682 ,  1.7226274 ,  2.9855597 ,
        3.1736856 ,  2.431657  ,  3.5203254 ,  0.9116562 ,  1.55

In [37]:
cosine_similarity(word_dict['장애'].reshape(1,-1), 
                  word_dict['장애등급제'].reshape(1,-1))

array([[0.82825625]], dtype=float32)

### Save model

In [46]:
model_gs.save("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/대학 관련/연세대학교/학회관련/YBIGTA/We are Team Analytics/2020-2/공모전/국회 입법 시각화/models/FasttextModel ver1.0 (iteration 5).bin")

### Load model

In [47]:
model_test = FastText.load("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/대학 관련/연세대학교/학회관련/YBIGTA/We are Team Analytics/2020-2/공모전/국회 입법 시각화/models/FasttextModel ver1.0 (iteration 5).bin")

In [48]:
model_test

In [49]:
word_dict_2 = {}
for vocab in tqdm(model_test.wv.index2word):
    word_dict_2[vocab] = model_test.wv[vocab]

In [50]:
cosine_similarity(word_dict_2['장애'].reshape(1,-1), 
                  word_dict_2['장애등급제'].reshape(1,-1))

array([[0.82825625]], dtype=float32)

# Train Fasttext (iteration 500)

In [51]:
%%time
model_gs = FastText()

model_gs.build_vocab(sentences = corpus)

CPU times: user 16.7 s, sys: 5.07 s, total: 21.8 s
Wall time: 23.1 s


In [ ]:
%%time
model_gs.train(sentences = corpus, total_examples = len(corpus), epochs=500)

### Save model

In [ ]:
model_gs.save("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/대학 관련/연세대학교/학회관련/YBIGTA/We are Team Analytics/2020-2/공모전/국회 입법 시각화/models/FasttextModel ver2.0 (iteration 500).bin")